In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Dense
from tensorflow.keras.layers import AvgPool2D, GlobalAveragePooling2D, MaxPool2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import ReLU, concatenate
import tensorflow.keras.backend as K
import numpy as np


In [2]:
test = np.zeros((212, 6, 340, 444))
with open("C:\\Users\\yangz\\Desktop\\FireProjectBatchDownloadNew\\DetectionDatasetFinal\\Testing_1696-1908.npy", 'rb') as f1:
        test = np.load(f1)
        

In [3]:
test = np.reshape(test, (212, 340, 444, 6))
yTest = np.zeros((212))
for i in range (212):
    if (i%2==0):
        yTest[i] = 0
    else:
        yTest[i] = 1


In [4]:
print (test.shape)
img_shape = (340, 444, 6)


(212, 340, 444, 6)


In [5]:
def denseNet(inputShape, nClasses, filters = 32):
    
    def BnRlCv (x,filters,kernel=1,strides=1):
        
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Conv2D(filters, kernel, strides=strides,padding = 'same')(x)
        return x
    
    def denseBlock(x, repetition):
        for _ in range(repetition):
            y = BnRlCv(x, 4*filters)
            y = BnRlCv(y, filters, 3)
            x = concatenate([y,x])
        return x
        
    def transition(x):
        x = BnRlCv(x, K.int_shape(x)[-1] //2 )
        x = AvgPool2D(2, strides = 2, padding = 'same')(x)
        return x
    
    input = Input (inputShape)
    x = Conv2D(64, 7, strides = 2, padding = 'same')(input)
    x = MaxPool2D(3, strides = 2, padding = 'same')(x)
    
    for repetition in [6,12,24,16]:
        d = denseBlock(x, repetition)
        x = transition(d)
    x = GlobalAveragePooling2D()(d)
    x = Dropout(0.3)(x)
    output = Dense(nClasses, activation = 'sigmoid')(x)
    
    model = Model(input, output)
    return model

inputShape = 340, 444, 6
nClasses = 1
model = denseNet(inputShape, nClasses)


In [17]:
model.load_weights("DenseNetTrainingB.hdf5")


In [18]:
yPred = model.predict(test)


In [19]:
yPred[yPred>=0.5] = 1
yPred[yPred<0.5] = 0
yPred = yPred.astype(int)


In [20]:
yPred = yPred.flatten()
print (yPred)


[0 0 0 0 0 0 0 0 0 1 0 1 1 1 0 1 0 1 0 1 0 1 1 1 0 1 0 1 0 1 0 1 0 0 0 1 0
 0 1 1 1 1 0 0 1 1 0 1 1 1 0 0 0 1 0 0 0 0 1 1 1 0 1 1 0 0 0 1 0 1 1 0 0 1
 0 0 1 1 0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 1 0 1 0 1 1 0 0 1 1 0 0 0 1 1 0
 0 0 0 1 1 0 1 0 1 0 1 0 0 0 1 0 1 0 0 0 1 0 1 0 1 0 1 0 0 0 1 0 1 0 1 0 1
 1 0 0 0 0 1 0 1 1 0 0 1 0 1 1 0 1 1 0 0 0 1 0 0 1 1 0 1 1 0 0 0 0 0 0 1 0
 1 1 1 0 0 0 0 1 1 0 1 0 1 0 1 0 1 0 1 0 0 0 1 0 0 1 0]


In [21]:
correct = 0
for i in range (212):
    if (yPred[i]==yTest[i]):
        correct+=1
        
print (correct/212)


0.6886792452830188
